In [24]:
from langchain_groq import ChatGroq

In [25]:

llm = ChatGroq(
    temperature=0, 
    groq_api_key='Your_API_KEY', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Highest point on this planet is?")
print(response.content)


The highest point on Earth is Mount Everest, located in the Himalayas on the border between Nepal and Tibet, China. It stands at a height of 8,848.86 meters (29,031.7 feet) above sea level.


In [26]:
#Now I'll use WebBaseLoader by Langchain this will help in webscrapping and collecting 
# info about job posting from company websites
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37070")
page_data = loader.load().pop().page_content
print(page_data)


Apply for Senior Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software EngineerKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does 

In [27]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [28]:
print(res.content)

```
{
  "role": "Senior Software Engineer",
  "experience": "5-8 years",
  "skills": [
    "Python",
    "Pyspark",
    "Databricks",
    "Delta lake",
    "Unity catalogue",
    "Kafka",
    "Airflow",
    "Spark",
    "AWS",
    "Dev Ops",
    "CI/CD automation",
    "Zero Touch deployments",
    "Design patterns",
    "Code reviews",
    "Git",
    "Docker",
    "Kubernetes",
    "JUnit",
    "TestNG",
    "Cucumber",
    "CI/CD",
    "Github Co-pilot"
  ],
  "description": "Design, develop and deliver high quality software solutions that solve specific business problems and grow Nike’s digital businesses. Be part of a mission critical team focused on building a high-quality, low-latency, fault-tolerant supply chain platform that drive Nike’s Digital transformation and growth strategies."
}
```


In [29]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res



{'role': 'Senior Software Engineer',
 'experience': '5-8 years',
 'skills': ['Python',
  'Pyspark',
  'Databricks',
  'Delta lake',
  'Unity catalogue',
  'Kafka',
  'Airflow',
  'Spark',
  'AWS',
  'Dev Ops',
  'CI/CD automation',
  'Zero Touch deployments',
  'Design patterns',
  'Code reviews',
  'Git',
  'Docker',
  'Kubernetes',
  'JUnit',
  'TestNG',
  'Cucumber',
  'CI/CD',
  'Github Co-pilot'],
 'description': 'Design, develop and deliver high quality software solutions that solve specific business problems and grow Nike’s digital businesses. Be part of a mission critical team focused on building a high-quality, low-latency, fault-tolerant supply chain platform that drive Nike’s Digital transformation and growth strategies.'}

In [30]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [31]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [32]:
type(json_res)

dict

In [34]:
job = json_res
job['skills']

['Python',
 'Pyspark',
 'Databricks',
 'Delta lake',
 'Unity catalogue',
 'Kafka',
 'Airflow',
 'Spark',
 'AWS',
 'Dev Ops',
 'CI/CD automation',
 'Zero Touch deployments',
 'Design patterns',
 'Code reviews',
 'Git',
 'Docker',
 'Kubernetes',
 'JUnit',
 'TestNG',
 'Cucumber',
 'CI/CD',
 'Github Co-pilot']

In [35]:
#Now I have the required skills based on this skill I'll query my vectordb to fetch relavent portfolio
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],


In [36]:
#Now based on the job and the portfolio's fetched I can generate a cold email that i can send to potentioal clients
#GNSV(Garimella Narasimha Sai Varun) is my madeup company please excuse the lack of creativity in naming it.😅
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description} 
        
        ### INSTRUCTION:
        You are Varun, a business development executive at GNSV. GNSV is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of GNSV
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase GNSV's portfolio: {link_list}
        Remember you are Varun, BDE at GNSV. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Solutions for Nike's Digital Transformation

Dear Hiring Manager,

I came across the Senior Software Engineer role at Nike, and I'm excited to introduce GNSV, an AI & Software Consulting company that can help you achieve your digital transformation goals. With our expertise in designing, developing, and delivering high-quality software solutions, we're confident that we can support Nike's mission-critical projects.

Our team has extensive experience in working with technologies like Python, Pyspark, Databricks, Delta lake, Unity catalogue, Kafka, Airflow, Spark, AWS, DevOps, CI/CD automation, Zero Touch deployments, Design patterns, Code reviews, Git, Docker, Kubernetes, JUnit, TestNG, Cucumber, and CI/CD. We've successfully implemented these technologies in various projects, ensuring scalability, process optimization, cost reduction, and heightened overall efficiency.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabi